In [34]:
#import STMprotocol
import apscheduler
from apscheduler.schedulers.background import BackgroundScheduler
import time

In [43]:
# %load STMprotocol.py
import serial
import struct
import datetime
import time

class STMprotocol:
    def __init__(self, serial_port):
        self.ser = serial.Serial(serial_port, 64000, timeout=0.2)
        self.pack_format = {
            0x01: "=BBBB",
            0x03: "=Bf",
            0x04: "=B",
            0x05: "=B",
            0x06: "=Bffff",
            0x07: "=B",
            0x08: "=fff",
            0x09: "=",
            0x0a: "=",
            0x0b: "=BH",
            0x0c: "=B",
            0x0d: "=B",
            0x0e: "=fff",
            0x0f: "=",            
            0xa0: "=",
            0xa1: "=B",
            0xb0: "=B",
            0xb1: "=B",
            0xb2: "=BB",
            0xb3: "=B",
            0xb4: "=B",
            0xb5: "=B",
            0xa2: "=ffffff",

        }

        self.unpack_format = {
            0x01: "=BBBB",
            0x03: "=BB",
            0x04: "=BB",
            0x05: "=BB",
            0x06: "=BB",
            0x07: "=ffff",
            0x08: "=BB",
            0x09: "=fff",
            0x0a: "=fff",
            0x0b: "=BB",
            0x0c: "=f",
            0x0d: "=BB",
            0x0e: "=BB",
            0x0f: "=fff",
            0xa0: "=B",
            0xa1: "=B",
            0xb0: "=BB",
            0xb1: "=BB",
            0xb2: "=BB",
            0xb3: "=BB",
            0xb4: "=BB",
            0xb5: "=BB",
            0xa2: "=BB",
        }

    def send_command(self, cmd, args):
        # Clear buffer
        #print(self.ser.read(self.ser.in_waiting))
        
        parameters = bytearray(struct.pack(self.pack_format[cmd], *args))
        #print(parameters)
        msg_len = len(parameters) + 5
        msg = bytearray([0xfa, 0xaf, msg_len, cmd]) + parameters
        crc = sum(msg) % 256
        msg += bytearray([crc])

        #print("send ", repr(msg))
        self.ser.write(msg)

        start_time = datetime.datetime.now()
        time_threshold = datetime.timedelta(seconds=1)
        dt = start_time - start_time
        
        time.sleep(0.001)
        data = self.ser.read()[0]
        while (data != 0xfa) and (dt < time_threshold):
            data = self.ser.read()[0]

            current_time = datetime.datetime.now()
            dt = start_time - current_time

        adr = self.ser.read()[0]
        answer_len = self.ser.read()[0]
        answer = bytearray(self.ser.read(answer_len - 3))
        #print("answer ", repr(bytearray([data, adr, answer_len]) + answer))

        args = struct.unpack(self.unpack_format[cmd], answer[1:-1])
        return args


In [66]:
class Greenhouse():
    stm_order = ["temperature","humidity","luminosity"]
    stm_get_command = [0x0F, []]
    def __init__(self, serial_port, read_time_interval_seconds=1):
        self.port = serial_port
        self.protocol = STMprotocol(self.port)
        #self.protocol = None
        self.read_time_interval = read_time_interval_seconds
        self.params = {
            'temperature' : 0,
            'humidity' : 0,
            'luminosity' : 0
        }
        self.goal_params = {
            'temperature': None,
            'humidity' : None,
            'luminosity' : None
        }
        self.history = []
        self.logfile = open("history"+str(time.time())+'.txt',"w+")
        self.scheduler = BackgroundScheduler()
        self.scheduler.add_job(self.update_params,'interval',seconds=self.read_time_interval)
        
    def convert_stm_return_to_dict(self, stm_array):
        return {param:value for param, value in zip(self.stm_order,stm_array)}
            
    def update_params(self, log=True):
        self.params = self.convert_stm_return_to_dict(self.protocol.send_command(*self.stm_get_command))
        if log:
            self.history.append((time.time(), self.params))
            self.logfile.write(str(time.time()) + ' ')
            print(time.time(),end=' ')
            for key in self.stm_order:
                self.logfile.write(str(self.params[key]) + ' ')
                print(self.params[key],end=' ')
            print('')
            self.logfile.write('\n')
            self.logfile.flush()
            
    def get_params(self):
        return self.params
    
    def set_goal_params(self, new_params):
        self.goal_params.update(new_params)
        #
    
    def start(self):
        self.scheduler.start()
    
    def stop(self):
        self.scheduler.shutdown()
        

In [68]:
g = Greenhouse("/dev/ttyUSB0")

In [69]:
g.update_params()

1521753926.8467813 24.450000762939453 98350.4609375 15.425999641418457 


In [67]:
g.protocol.ser.close()

In [70]:
g.start()

1521753997.2987618 24.469999313354492 98353.0625 15.529999732971191 
1521753998.29776 24.469999313354492 98352.5703125 15.189000129699707 
1521753999.2983632 24.469999313354492 98353.4765625 15.371000289916992 
1521754000.2984672 24.469999313354492 98353.4765625 14.086000442504883 
1521754001.298746 24.469999313354492 98353.8046875 15.291000366210938 
1521754002.2979698 24.469999313354492 98353.8046875 15.689000129699707 
1521754003.29827 24.469999313354492 98353.4765625 15.472999572753906 
1521754004.2983613 24.469999313354492 98352.8203125 15.097999572753906 
1521754005.298594 24.479999542236328 98353.40625 15.234999656677246 
1521754006.29891 24.469999313354492 98353.1484375 15.484999656677246 
1521754007.2991848 24.479999542236328 98352.75 15.564000129699707 
1521754008.2984395 24.479999542236328 98351.8515625 15.722999572753906 
1521754009.297266 24.479999542236328 98351.5234375 15.371000289916992 
1521754010.2972853 24.479999542236328 98351.2890625 15.734999656677246 
1521754011.

1521754116.297807 24.540000915527344 98351.375 15.770999908447266 
1521754117.2980053 24.540000915527344 98351.2890625 14.815999984741211 
1521754118.297093 24.540000915527344 98350.390625 14.305000305175781 
1521754119.2975218 24.540000915527344 98350.3046875 15.498000144958496 
1521754120.2976599 24.540000915527344 98350.5625 14.805000305175781 
1521754121.2979212 24.540000915527344 98351.046875 15.793000221252441 
1521754122.2968895 24.540000915527344 98351.3046875 15.71399974822998 
1521754123.2969291 24.540000915527344 98350.8046875 15.24899959564209 
1521754124.2969227 24.549999237060547 98350.734375 14.487000465393066 
1521754125.2970662 24.549999237060547 98351.2265625 15.350000381469727 
1521754126.2975373 24.549999237060547 98351.1484375 15.270999908447266 
1521754127.297758 24.549999237060547 98350.9921875 14.987000465393066 
1521754128.298145 24.549999237060547 98349.5859375 14.508999824523926 
1521754129.2978933 24.549999237060547 98348.359375 14.930000305175781 
152175413

1521754233.2975104 24.559999465942383 98345.6796875 14.532999992370605 
1521754234.2975185 24.559999465942383 98345.3515625 15.08899974822998 
1521754235.2974927 24.559999465942383 98345.6796875 14.329000473022461 
1521754236.2979944 24.559999465942383 98345.6015625 15.680000305175781 
1521754237.2979815 24.559999465942383 98345.34375 14.383999824523926 
1521754238.297318 24.559999465942383 98344.5234375 14.99899959564209 
1521754239.29757 24.549999237060547 98345.5859375 14.942000389099121 
1521754240.297755 24.549999237060547 98346.2421875 14.85099983215332 
1521754241.2979677 24.549999237060547 98346.734375 14.62399959564209 
1521754242.297946 24.549999237060547 98346.5546875 14.692000389099121 
1521754243.2970982 24.549999237060547 98345.8046875 14.600000381469727 
1521754244.2974415 24.549999237060547 98345.484375 15.395999908447266 
1521754245.2973835 24.549999237060547 98345.484375 14.531999588012695 
1521754246.2975 24.549999237060547 98346.140625 15.350000381469727 
1521754247

1521754350.2974591 24.520000457763672 98346.359375 15.645000457763672 
1521754351.2974546 24.520000457763672 98346.03125 15.123000144958496 
1521754352.297426 24.520000457763672 98346.359375 14.791999816894531 
1521754353.2977662 24.520000457763672 98346.2890625 14.748000144958496 
1521754354.2979696 24.520000457763672 98345.953125 15.111000061035156 
1521754355.296945 24.520000457763672 98347.1015625 14.781000137329102 
1521754356.297338 24.520000457763672 98347.1796875 14.303999900817871 
1521754357.2975624 24.520000457763672 98347.5078125 15.236000061035156 
1521754358.2976575 24.520000457763672 98348.9921875 14.8149995803833 
1521754359.2980084 24.520000457763672 98349.3125 14.873000144958496 
1521754360.2969813 24.520000457763672 98350.1328125 14.258000373840332 
1521754361.2970634 24.520000457763672 98348.65625 15.1899995803833 
1521754362.2973526 24.520000457763672 98348.9921875 15.395000457763672 
1521754363.2975926 24.520000457763672 98348.15625 14.986000061035156 
1521754364.

1521754466.2968757 24.510000228881836 98344.4609375 15.769000053405762 
1521754467.2981603 24.510000228881836 98344.4609375 14.746999740600586 
1521754468.2969043 24.510000228881836 98345.2890625 15.178000450134277 
1521754469.2970362 24.510000228881836 98344.046875 15.586999893188477 
1521754470.2975287 24.510000228881836 98343.234375 15.008000373840332 
1521754471.2975774 24.510000228881836 98342.40625 14.9399995803833 
1521754472.2977989 24.510000228881836 98342.40625 14.916999816894531 
1521754473.2979946 24.510000228881836 98342.40625 15.4399995803833 
1521754474.2972193 24.510000228881836 98341.09375 15.246999740600586 
1521754475.297454 24.510000228881836 98341.3359375 15.031000137329102 
1521754476.2977717 24.510000228881836 98342.8125 14.531000137329102 
1521754477.2981281 24.510000228881836 98344.2890625 14.598999977111816 
1521754478.2971542 24.510000228881836 98343.8046875 15.020000457763672 
1521754479.297128 24.510000228881836 98343.6484375 15.098999977111816 
1521754480.

...

1521755063.2971096 24.459999084472656 98338.015625 15.324999809265137 
1521755064.2974901 24.459999084472656 98336.2890625 14.847000122070312 
1521755065.2977362 24.459999084472656 98337.03125 14.746000289916992 
1521755066.2980745 24.459999084472656 98336.8671875 14.972000122070312 
1521755067.2972236 24.459999084472656 98336.5390625 15.824999809265137 
1521755068.297372 24.459999084472656 98336.1171875 14.927000045776367 
1521755069.2977824 24.459999084472656 98336.375 15.404000282287598 
1521755070.2979693 24.459999084472656 98336.703125 14.654999732971191 
1521755071.2970195 24.459999084472656 98336.2890625 15.496000289916992 
1521755072.2973883 24.450000762939453 98335.21875 14.734000205993652 
1521755073.297595 24.450000762939453 98335.875 15.859000205993652 
1521755074.2978592 24.450000762939453 98336.203125 15.472000122070312 
1521755075.2968996 24.450000762939453 98334.6328125 15.812999725341797 
1521755076.2972815 24.450000762939453 98336.6015625 14.949999809265137 
152175507

1521755179.2973514 24.440000534057617 98324.8984375 14.494999885559082 
1521755180.2975361 24.440000534057617 98324.796875 15.347000122070312 
1521755181.2978127 24.440000534057617 98324.640625 15.607999801635742 
1521755182.2980518 24.440000534057617 98324.5703125 15.654000282287598 
1521755183.2971375 24.440000534057617 98324.5703125 14.437999725341797 
1521755184.2976034 24.440000534057617 98325.453125 14.699999809265137 
1521755185.2977579 24.43000030517578 98324.1328125 14.267000198364258 
1521755186.2978175 24.43000030517578 98323.96875 14.744999885559082 
1521755187.2972817 24.43000030517578 98324.1328125 14.675999641418457 
1521755188.297382 24.43000030517578 98324.2109375 15.029000282287598 
1521755189.2974446 24.43000030517578 98323.4609375 15.369999885559082 
1521755190.297689 24.43000030517578 98324.453125 14.892000198364258 
1521755191.2967594 24.43000030517578 98325.2734375 14.789999961853027 
1521755192.297009 24.43000030517578 98324.859375 14.595999717712402 
1521755193

1521755296.297827 24.43000030517578 98330.2734375 15.289999961853027 
1521755297.29697 24.420000076293945 98329.6875 15.994000434875488 
1521755298.2973306 24.420000076293945 98330.5859375 15.800000190734863 
1521755299.2976809 24.420000076293945 98329.515625 15.425000190734863 
1521755300.2977507 24.420000076293945 98331.40625 14.652999877929688 
1521755301.2969368 24.420000076293945 98330.6640625 15.595999717712402 
1521755302.2973425 24.420000076293945 98331.8046875 14.675000190734863 
1521755303.297663 24.420000076293945 98331.71875 15.675000190734863 
1521755304.2976458 24.40999984741211 98331.140625 15.175000190734863 
1521755305.2977145 24.40999984741211 98331.6328125 14.833999633789062 
1521755306.2976978 24.40999984741211 98331.9609375 14.824000358581543 
1521755307.2977562 24.40999984741211 98332.2890625 14.857000350952148 
1521755308.297758 24.40999984741211 98332.0390625 14.937000274658203 
1521755309.2981327 24.40999984741211 98331.0546875 14.505000114440918 
1521755310.29

1521755413.2978926 24.440000534057617 98322.71875 14.369999885559082 
1521755414.2982519 24.440000534057617 98322.5546875 15.097000122070312 
1521755415.2973752 24.440000534057617 98322.8828125 14.550999641418457 
1521755416.2977185 24.440000534057617 98323.375 14.767000198364258 
1521755417.2978127 24.440000534057617 98323.4140625 15.232999801635742 
1521755418.2980354 24.440000534057617 98323.0859375 14.880999565124512 
1521755419.2973487 24.440000534057617 98322.7578125 15.505999565124512 
1521755420.297657 24.440000534057617 98323.9140625 15.017000198364258 
1521755421.2978587 24.440000534057617 98324.5546875 15.494999885559082 
1521755422.2981117 24.440000534057617 98324.0546875 15.324000358581543 
1521755423.297146 24.440000534057617 98323.2421875 15.414999961853027 
1521755424.2976174 24.440000534057617 98323.40625 15.630000114440918 
1521755425.2977276 24.440000534057617 98323.2421875 14.949000358581543 
1521755426.2979417 24.43000030517578 98322.9921875 15.664999961853027 
152

1521755530.2979038 24.40999984741211 98328.4296875 15.208999633789062 
1521755531.296867 24.40999984741211 98328.4296875 14.630000114440918 
1521755532.2973542 24.40999984741211 98326.8671875 15.550000190734863 
1521755533.2976596 24.40999984741211 98328.0078125 15.458999633789062 
1521755534.297868 24.40999984741211 98328.3359375 14.937000274658203 
1521755535.2968795 24.40999984741211 98328.0078125 15.312000274658203 
1521755536.297289 24.40999984741211 98326.8671875 14.640999794006348 
1521755537.2976494 24.40999984741211 98327.4296875 14.687000274658203 
1521755538.2977538 24.40999984741211 98326.9375 15.300000190734863 
1521755539.2980494 24.40999984741211 98326.4453125 14.675000190734863 
1521755540.2971265 24.40999984741211 98327.0234375 15.187000274658203 
1521755541.2974741 24.40999984741211 98327.3515625 15.970000267028809 
1521755542.297778 24.40999984741211 98326.5390625 14.414999961853027 
1521755543.2979672 24.40999984741211 98327.0234375 16.322999954223633 
1521755544.29

In [71]:
g.stop()

1521755576.2983546 24.40999984741211 98327.4296875 15.913999557495117 
